In [1]:
import torch
from torch import nn
import torchviz

import vit
import scipy.io
from matplotlib import pyplot as plt
import numpy as np
import pandas as pd
from importlib import reload
from mat import mat

/home/amintehrani/.pyenv/versions/3.12.8/envs/torchgeo/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
mat.keys()

dict_keys(['__header__', '__version__', '__globals__', 'all_images', 'all_dnas', 'all_labels', 'all_dnas_norepeat', 'all_dna_labels_norepeat', 'all_boldids', 'train_loc', 'val_seen_loc', 'val_unseen_loc', 'test_seen_loc', 'test_unseen_loc', 'species2genus', 'described_species_labels_train', 'described_species_labels_trainval', 'all_dna_features_cnn_original', 'all_image_features_resnet', 'all_image_features_gan', 'all_dna_features_cnn_new', 'all_string_dnas'])

In [3]:
species2genus = mat['species2genus']-1

# group species by genus

genus_species = dict()
max_specie_in_genus = 0
for genus_id, genus in pd.DataFrame(species2genus, columns=['genus']).groupby('genus'):
    specie_indices = genus.index.tolist()
    genus_species[genus_id] = specie_indices
    if len(specie_indices) > max_specie_in_genus:
        max_specie_in_genus = len(specie_indices)

print(len(genus_species))
print("Max specie in genus: ", max_specie_in_genus)


372
Max specie in genus:  23


In [4]:
from sklearn.decomposition import PCA

pca = PCA(n_components=512)
all_dna_features_cnn_pca = np.array(pca.fit_transform(mat['all_dna_features_cnn_new']))


KeyboardInterrupt: 

In [4]:
pca = PCA(n_components=768)
all_image_features_gan_pca = np.array(pca.fit_transform(mat['all_image_features_gan']))

In [5]:

all_dna_len = list(map(lambda s: len(s.strip()), mat['all_string_dnas']))
dna_str_len_mapping: dict[int,int] = {}

def dna_str_len_to_int(s_len):
    if s_len not in dna_str_len_mapping:
        dna_str_len_mapping[s_len] = len(dna_str_len_mapping)
    return dna_str_len_mapping[s_len]

# def all_dna_len_token():
#     return list(map(dna_str_len_to_int, all_dna_len))

all_dna_len_tokens = list(map(dna_str_len_to_int, all_dna_len))
all_dna_len_tokens = np.array(all_dna_len_tokens, dtype=np.int64)
print(list(zip(all_dna_len, all_dna_len_tokens))[:20])

[(658, np.int64(0)), (658, np.int64(0)), (658, np.int64(0)), (658, np.int64(0)), (658, np.int64(0)), (658, np.int64(0)), (658, np.int64(0)), (658, np.int64(0)), (658, np.int64(0)), (658, np.int64(0)), (658, np.int64(0)), (658, np.int64(0)), (658, np.int64(0)), (658, np.int64(0)), (658, np.int64(0)), (658, np.int64(0)), (658, np.int64(0)), (658, np.int64(0)), (658, np.int64(0)), (658, np.int64(0))]


In [ ]:
deviceGPU = torch.device("cuda" if torch.cuda.is_available() else "cpu")
deviceCPU = torch.device("cpu")

device = deviceGP
device

device(type='cpu')

In [7]:
import vit
reload(vit)
from vit import get_processor_encoder, get_img_embedding
img_processor, img_encoder = get_processor_encoder("./vit-finetuned7-final", device)
get_img_embedding(mat['all_images'][:2], img_processor, img_encoder, device).shape

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.50, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.
Some weights of ViTModel were not initialized from the model checkpoint at ./vit-finetuned7-final and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


torch.Size([2, 768])

In [8]:
import dnaencoder
reload(dnaencoder)
from dnaencoder import get_tokenizer_encoder, get_dna_embedding
dna_tokenizer, dna_encoder = get_tokenizer_encoder("./dnaencoder-finetuned1755100772-final", deviceGPU)
get_dna_embedding(mat['all_string_dnas'][:2], dna_tokenizer, dna_encoder).shape

<class 'numpy.ndarray'>


torch.Size([2, 512])

In [ ]:
# import multimodal_dataset
# reload(multimodal_dataset)
# from multimodal_dataset import MultiModalDataset
# dataset = MultiModalDataset(mat['all_string_dnas'], mat['all_images'], np.transpose(mat['all_labels'], (1,0)), dna_str_len_mapping, species2genus, genus_species, None, None, )
#                             # dna_embeddings=all_dna_features_cnn_pca, img_embeddings=all_image_features_gan_pca)
# (dataset[0])
# _ = (dataset[0:5])




In [10]:
train_indices = (mat['train_loc'] - 1).flatten()  # Get train indices
val_indices = (mat['val_seen_loc'] - 1).flatten()  # Get validation indices



In [12]:
import multimodal_dataset
reload(multimodal_dataset)
from multimodal_dataset import MultiModalDataset
# all_dataset = MultiModalDataset(mat['all_string_dnas'], mat['all_images'], np.transpose(mat['all_labels'], (1,0)), dna_str_len_mapping, species2genus, genus_species, None, None, 
#                             dna_embeddings=all_dna_features_cnn_pca, img_embeddings=all_image_features_gan_pca)

train_dataset = MultiModalDataset(mat['all_string_dnas'][train_indices], mat['all_images'][train_indices], np.transpose(mat['all_labels'], (1,0))[train_indices], dna_str_len_mapping, species2genus, genus_species, img_processor, dna_tokenizer,)
val_dataset = MultiModalDataset(mat['all_string_dnas'][val_indices], mat['all_images'][val_indices], np.transpose(mat['all_labels'], (1,0))[val_indices], dna_str_len_mapping, species2genus, genus_species, img_processor, dna_tokenizer,)

In [15]:


import models
reload(models)
from models import AttentionFusion, GenusClassifier, LocalSpecieClassfier, MainClassifier, multimodal_collector
from transformers import DefaultDataCollator


def get_main_classifier():
    fusion_embedder = AttentionFusion(dna_dim=512,img_dim=768,dna_len_dim=16)
    print("Fusion model created. fused dim: ", fusion_embedder.fused_dim)
    genus_classifier = GenusClassifier(fusion_embedder.fused_dim)

    local_specie_classifier = LocalSpecieClassfier(fusion_embedder.fused_dim)

    return MainClassifier(mat['species2genus'], genus_species, dna_encoder, img_encoder, fusion_embedder, genus_classifier, local_specie_classifier).to(device)

main_classifier = get_main_classifier()

# print(main_classifier(**multimodal_collector([dataset[0], dataset[1]])))
# print(main_classifier(**dataset[0]))
# print(main_classifier(**dataset[0:2]))
import os
# os.environ["CUDA_LAUNCH_BLOCKING"] = "1"  # Disable Weights & Biases logging
# os.environ["TORCH_USE_CUDA_DSA"] = "1"  # Disable Weights & Biases logging


Fusion model created. fused dim:  1296


In [16]:
main_classifier.fit(
    train_dataset,
    val_dataset,
    batch_size=32,
    epochs=3,
    eval_steps=10,
)

OutOfMemoryError: CUDA out of memory. Tried to allocate 4.88 GiB. GPU 0 has a total capacity of 1.95 GiB of which 330.56 MiB is free. Including non-PyTorch memory, this process has 1.62 GiB memory in use. Of the allocated memory 1.54 GiB is allocated by PyTorch, and 47.66 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)